In [0]:
import sys
import os

# 1. Configura Path para importar 'src'
repo_root = os.getcwd()
repo_root = '/'.join(repo_root.split('/')[:-1])
if repo_root not in sys.path:
    sys.path.append(repo_root)



In [0]:
%pip install -r ../requirements.txt

In [0]:

from src.utility.environment import Environment
from src.infrastructure.data_manager import DataManager

# Baixando os dados do kaggle

In [0]:
os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(Environment.KAGGLE_CONFIG_DIR)
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()



In [0]:
print(f"⬇️ Baixando dataset: {Environment.kaggle_dataset_name}...")
api.dataset_download_files(
    Environment.kaggle_dataset_name, 
    path=Environment.DATABRICKS_CATALOG, 
    unzip=True
)
print("✅ Download concluído.")

# Criado tabelas dos arquivos

In [0]:
manager = DataManager(spark)

path_prefix = Environment.DATABRICKS_CATALOG



In [0]:
files = [f for f in os.listdir(path_prefix) if f.endswith('.csv')]

for file in files:
    table_name = file.split('.')[0]
    df = manager.read_csv(f"{path_prefix}/{file}")
    df.write\
    .format("delta")\
    .mode("overwrite")\
    .option("overwriteSchema", "true")\
    .saveAsTable(f"kaggle.{table_name}")